In [2]:
import pandas as pd
from os import listdir
import json
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from torch import from_numpy, stack, cat, FloatTensor, save
from torch.nn import functional as F

In [3]:
DATA_FOLDER = "./data/"
OUTPUT_FOLDER = "./data/"

In [4]:
inference_args = {"selected_columns": ["x_left_hand_0", "x_left_hand_1", "x_left_hand_2", "x_left_hand_3", "x_left_hand_4", "x_left_hand_5", "x_left_hand_6", "x_left_hand_7", "x_left_hand_8", "x_left_hand_9", "x_left_hand_10", "x_left_hand_11", "x_left_hand_12", "x_left_hand_13", "x_left_hand_14", "x_left_hand_15", "x_left_hand_16", "x_left_hand_17", "x_left_hand_18", "x_left_hand_19", "x_left_hand_20", "y_left_hand_0", "y_left_hand_1", "y_left_hand_2", "y_left_hand_3", "y_left_hand_4", "y_left_hand_5", "y_left_hand_6", "y_left_hand_7", "y_left_hand_8", "y_left_hand_9", "y_left_hand_10", "y_left_hand_11", "y_left_hand_12", "y_left_hand_13", "y_left_hand_14", "y_left_hand_15", "y_left_hand_16", "y_left_hand_17", "y_left_hand_18", "y_left_hand_19", "y_left_hand_20", "x_right_hand_0", "x_right_hand_1", "x_right_hand_2", "x_right_hand_3", "x_right_hand_4", "x_right_hand_5", "x_right_hand_6", "x_right_hand_7", "x_right_hand_8", "x_right_hand_9", "x_right_hand_10", "x_right_hand_11", "x_right_hand_12", "x_right_hand_13", "x_right_hand_14", "x_right_hand_15", "x_right_hand_16", "x_right_hand_17", "x_right_hand_18", "x_right_hand_19", "x_right_hand_20", "y_right_hand_0", "y_right_hand_1", "y_right_hand_2", "y_right_hand_3", "y_right_hand_4", "y_right_hand_5", "y_right_hand_6", "y_right_hand_7", "y_right_hand_8", "y_right_hand_9", "y_right_hand_10", "y_right_hand_11", "y_right_hand_12", "y_right_hand_13", "y_right_hand_14", "y_right_hand_15", "y_right_hand_16", "y_right_hand_17", "y_right_hand_18", "y_right_hand_19", "y_right_hand_20", "x_face_0", "x_face_13", "x_face_14", "x_face_17", "x_face_37", "x_face_39", "x_face_40", "x_face_61", "x_face_78", "x_face_80", "x_face_81", "x_face_82", "x_face_84", "x_face_87", "x_face_88", "x_face_91", "x_face_95", "x_face_146", "x_face_178", "x_face_181", "x_face_185", "x_face_191", "x_face_267", "x_face_269", "x_face_270", "x_face_291", "x_face_308", "x_face_310", "x_face_311", "x_face_312", "x_face_314", "x_face_317", "x_face_318", "x_face_321", "x_face_324", "x_face_375", "x_face_402", "x_face_405", "x_face_409", "x_face_415", "y_face_0", "y_face_13", "y_face_14", "y_face_17", "y_face_37", "y_face_39", "y_face_40", "y_face_61", "y_face_78", "y_face_80", "y_face_81", "y_face_82", "y_face_84", "y_face_87", "y_face_88", "y_face_91", "y_face_95", "y_face_146", "y_face_178", "y_face_181", "y_face_185", "y_face_191", "y_face_267", "y_face_269", "y_face_270", "y_face_291", "y_face_308", "y_face_310", "y_face_311", "y_face_312", "y_face_314", "y_face_317", "y_face_318", "y_face_321", "y_face_324", "y_face_375", "y_face_402", "y_face_405", "y_face_409", "y_face_415"]}
with open(OUTPUT_FOLDER + "inference_args.json", "w") as f: json.dump(inference_args, f)

In [ ]:
file_paths = listdir(f"{DATA_FOLDER}train_landmarks/")
print(len(file_paths))

In [ ]:
with open(OUTPUT_FOLDER + "inference_args.json", "r") as f:
    selected_columns = json.load(f)["selected_columns"]
print(len(selected_columns))

In [ ]:
train_labels = pd.read_csv(DATA_FOLDER + "train.csv").set_index("sequence_id").loc[:, "phrase"].to_dict()
train_labels

In [ ]:
class CTP():
    
    def __init__(self):
        super(CTP, self).__init__()
        
        with open (DATA_FOLDER + "character_to_prediction_index.json", "r") as f: self.character_map = json.load(f)
        self.rev_character_map = {j:i for i,j in self.character_map.items()}
        
    def string_to_list_pred(self, string):
        return [self.char_to_pred(char) for char in string]
        
    def list_pred_to_string(self, list_pred):
        return "".join([self.pred_to_char(p) for p in list_pred])
        
    def char_to_pred(self, char):
        return self.character_map[char]
    
    def pred_to_char(self, pred):
        return self.rev_character_map[pred]

In [ ]:
def get_subsets(path):
    df = pd.read_parquet(path, columns=selected_columns)
    sequence_ids = df.index.unique().to_list()
    for seq in sequence_ids:
        x = df.loc[seq]
        if type(x) == pd.Series: x = x.to_frame().T
        
        y = train_labels[seq]
        yield x,y

In [ ]:
dfs_lengths = []
for path in tqdm(file_paths):
    dfs_lengths += [len(df) for df, label in get_subsets(DATA_FOLDER + "train_landmarks/" + path)]

In [ ]:
plt.figure(figsize=(10,5))
hist = plt.hist(dfs_lengths, bins=200)

In [ ]:
MIN_FRAMES = 50
MAX_FRAMES = 250

In [ ]:
max([len(phrase) for phrase in train_labels.values()])

In [ ]:
MAX_PHRASE = 31 + 1

In [ ]:
ctp = CTP()
xs,ys = [], []
for path in tqdm(file_paths):
    for x, y in get_subsets(DATA_FOLDER + "train_landmarks/" + path):
        if not (MAX_FRAMES >= x.shape[0] >= MIN_FRAMES): continue
        x = F.pad(from_numpy(x.to_numpy()), (0,0,0,MAX_FRAMES - x.shape[0]), "constant", 0)
        y = ctp.string_to_list_pred(y)
        y = FloatTensor(y + [60] + [61 for _ in range(MAX_PHRASE - 1 - len(y))])
        xs.append(x); ys.append(y)

In [ ]:
print(len(xs), len(train_labels))

In [ ]:
save(xs, OUTPUT_FOLDER + "x.torch")
save(ys, OUTPUT_FOLDER + "y.torch")